# 1.1 - Series Temporales

![time_series](images/time_series.jpg)

In [ ]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import pylab as plt

In [ ]:
df=pd.read_csv('../data/occupancy.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df.date = pd.to_datetime(df.date)

df.head()

In [ ]:
df.info()

In [ ]:
# pasar la fecha al indice

df = df.set_index('date')

df.head()

In [ ]:
df.CO2.plot();

In [ ]:
df.Light.plot();

In [ ]:
df.Temperature.plot();

In [ ]:
dias = df.CO2.loc['2015-4-1': '2015-4-30']

dias.plot();

### Autoregresión - AR

El modelo usa la relación de dependencia del momento actual con $n$ momentos anteriores. Es, en realidad, una regresión lineal.

$$y_t = \beta_0+\beta_1y_{t-1}+\beta_2y_{t-2}+\ldots+\beta_ny_{t-n}+\epsilon_t$$

Si solamente nos fijamos en el momento anterior, $t-1$, entonces:

$$y_t = \beta_0+\beta_1y_{t-1}+\epsilon_t$$

Es la llamada Cadena de Markov

In [ ]:
from pandas.plotting import lag_plot

In [ ]:
lag_plot(df.CO2)

In [ ]:
# datos random

lag_plot(pd.Series(np.random.random(1000)));

In [ ]:
# AR

from statsmodels.tsa.ar_model import AutoReg as AR

In [ ]:
train, test = df.CO2[:-10], df.CO2[-10:]

train.shape, test.shape

In [ ]:
modelo = AR(train, lags=1).fit()

# lags=1 es la cadena de Markov, lags son los pasos atras que mira

In [ ]:
len(train), len(df.CO2)-1  

In [ ]:
# por indice para predecir

pred = modelo.predict(len(train), len(df.CO2)-1)

len(pred)

In [ ]:
pred

In [ ]:
test

In [ ]:
# error medio absoluto MAE


error = (pred - test).abs().sum()/len(pred)  

error

In [ ]:
res=pd.DataFrame({'real': test, 'pred': pred, 'error': pred - test})

res

In [ ]:
import warnings
warnings.simplefilter('ignore')

for i in range(10, 20, 1):
    
    modelo=AR(train, lags=i).fit()                     # entrenar la AR
    
    pred=modelo.predict(len(train), len(df.CO2)-1)     # la prediccion en el intervalo
    
    error=(pred-test).abs().sum()/len(pred)            # error medio absoluto
    
    print ('Error mae: ', error.round(5), '  Maxlag : ', i)

### Media Móvil (Moving Average MA)

Una manera matemática de expresar la MA puede ser:

$$y_t = \beta_0+\epsilon_t+\epsilon_{t-1}\phi_{t-1}+\epsilon_{t-2}\phi_{t-2}+\ldots+\epsilon_{t-n}\phi_{t-n}$$


Se usa la relación entre las observaciones y el error residual de la media móvil.

Otra es la evidente, la media móvil, se muestra en el gráfico.

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(dias.values)

plt.plot([50, 80],[1000, 1000], color='black')
plt.plot([50, 50],[1000, 1100], color='black')
plt.plot([50, 80],[1100, 1100], color='black')
plt.plot([80, 80],[1000, 1100], color='black')

plt.quiver(80, 1050, 1, 0, scale=20, color='r');

In [ ]:
import statsmodels.api as sm

res=sm.tsa.seasonal_decompose(dias)

resplot=res.plot()

### ARMA (AutoRegressive Moving Average)

Unión de ambos conceptos.

$$y_t = \beta_0 + \epsilon_t + \sum_{i=1}^{n}\beta_iy_{t-i} + \sum_{i=1}^{m}\phi_i\epsilon_{t-i}$$

In [ ]:
# ARIMA  order==> (p, d, q)  d=>Integrated , d=0 es ARMA

# ARMA  order==> p=>AR (n, maxlag), q=>MA (m, ancho de la ventana)

from statsmodels.tsa.arima.model import ARIMA

In [ ]:
%%time

modelo=ARIMA(train, order=(16, 0, 3)).fit()  # d=0 implica ARMA

In [ ]:
pred=modelo.predict(len(train), len(df.CO2)-1) 

error=(pred-test).abs().sum()/len(pred) 

print ('Error mae: ', error)

res=pd.DataFrame({'real':test, 'pred':pred, 'error':pred-test})

res

### ARIMA (AutoRegressive Integrated Moving Average)

Lo de Integrated trata de hacer la serie temporal estacionaria, similar al concepto de PID.

In [ ]:
%%time

modelo=ARIMA(train, order=(4, 1, 2)).fit()

                     
pred=modelo.predict(len(train), len(df.CO2)-1) 

error=(pred-test).abs().sum()/len(pred) 

print ('Error mae: ', error)

res=pd.DataFrame({'real':test, 'pred':pred, 'error':pred-test})
res

### ADF-test (testeo de la estacionaridad)

https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html

### SARIMA (Seasonal AutoRegressive Integrated Moving Average)
Arima por estaciones.

### SARIMAX (Seasonal AutoRegressive Integrated Moving Average Exogenous)
SARIMA con regresores exógenos.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
#help(SARIMAX)

In [ ]:
%%time

modelo=SARIMAX(endog=train, 
               exog=None,
               #order=(4, 1, 5),  # modelo arima
               seasonal_order=(4, 1, 5, 2) # modelo sarima, estacionalidad es el ultimo
              ).fit(disp=False)
                     
pred=modelo.predict(len(train), len(df.CO2)-1)  

error=(pred-test).abs().sum()/len(pred) 
print ('Error mae: ', error)


res=pd.DataFrame({'real':test, 'pred':pred, 'error':pred-test})
res

## FB-Prophet

https://facebook.github.io/prophet/docs/quick_start.html

In [ ]:
%pip install prophet

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from fbprophet import Prophet


from sklearn.metrics import mean_squared_error, mean_absolute_error

plt.style.use('fivethirtyeight')

In [ ]:
# datos, consumo de energia

pjme=pd.read_csv('../data/PJME_hourly.csv', index_col=[0], parse_dates=[0])
pjme.tail()

In [ ]:
_=pjme.plot(style='.', figsize=(15, 5), color='r', title='PJME')

In [ ]:
# se definen las caracteristicas temporales

def features(df, label):
    
    df=df.copy()
    
    df['date']=df.index
    df['hour']=df.date.dt.hour
    df['day_of_week']=df.date.dt.dayofweek
    df['quarter']=df.date.dt.quarter
    df['month']=df.date.dt.month
    df['year']=df.date.dt.year
    df['day_of_year']=df.date.dt.dayofyear
    df['day_of_month']=df.date.dt.day
    df['week_of_year']=df.date.dt.weekofyear
    
    X=df[['hour', 'day_of_week', 'quarter', 'month', 'year',
          'day_of_year', 'day_of_month', 'week_of_year']]
    y=df[label]
    
    return X,y

In [ ]:
X, y = features(pjme, 'PJME_MW')

data=pd.concat([X, y], axis=1)

data.head()

In [ ]:
sns.pairplot(data.dropna(), 
             hue='hour', 
             x_vars=['hour', 'day_of_week', 'year', 'week_of_year'],
             y_vars='PJME_MW', 
             height=5, plot_kws={'alpha':0.1, 'linewidth':0})

plt.suptitle('MW por hora, dia, dia de la semana, y semana del año')
plt.show();

In [ ]:
split_date='01-Jan-2015'

train=pjme.loc[pjme.index<=split_date].copy()

test=pjme.loc[pjme.index>split_date].copy()

_=test.rename(columns={'PJME_MW': 'Test'})\
      .join(train.rename(columns={'PJME_MW': 'Train'}), how='outer')\
      .plot(figsize=(15,5), title='PJME', style='.')

In [ ]:
# IMPORTANTE, fbprophet necesita estos nombres de columnas

train=train.reset_index().rename(columns={'Datetime':'ds', 'PJME_MW':'y'})

test=test.reset_index().rename(columns={'Datetime':'ds', 'PJME_MW':'y'})

train.head()

In [ ]:
test.head()

In [ ]:
%%time

modelo=Prophet().fit(train)

pred=modelo.predict(test)

In [ ]:
f, ax=plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

fig=modelo.plot(pred, ax=ax)

In [ ]:
# comparando

f, ax=plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test.ds, test.y, color='r')
fig=modelo.plot(pred, ax)

In [ ]:
# metricas de error

mean_squared_error(test.y, pred.yhat)**0.5

In [ ]:
# Con vacaciones

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()

train_h = cal.holidays(train.index.min(), train.index.max())

test_h = cal.holidays(test.index.min(), test.index.max())

In [ ]:
pjme['date'] = pjme.index.date

pjme['is_holiday'] = pjme.date.isin([d.date() for d in cal.holidays()])

df_h = pjme.loc[pjme['is_holiday']].reset_index().rename(columns={'Datetime':'ds'})

df_h['holiday'] = 'USFederalHoliday'

df_h = df_h.drop(['PJME_MW','date','is_holiday'], axis=1)

df_h.head()

In [ ]:
modelo_h=Prophet(holidays=df_h).fit(train)

fig2=modelo_h.plot_components(pred)

In [ ]:
pred_h=modelo_h.predict(test)

pred_h.head()

In [ ]:
mean_squared_error(test.y, pred_h.yhat, squared=False)

### Yahoo Finance

In [ ]:
%pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf

import time

In [ ]:
data=yf.download(tickers='AAPL', period='5d', interval='1m')

data['datetime']=data.index

data.to_dict(orient='records')[-1]

In [ ]:
data=yf.download(tickers='UBER', period='5d', interval='1m')

data['datetime']=data.index

data.to_dict(orient='records')[-1]

In [ ]:
data=yf.download(tickers='^DJI', period='5d', interval='1m')

data['datetime']=data.index

data.to_dict(orient='records')[-1]

In [ ]:
data=yf.download(tickers='^IBEX', period='5d', interval='1m')

data['datetime']=data.index

data.tail()

In [ ]:
while 1:
    data=yf.download(tickers='^IBEX', period='5d', interval='1m')

    data['datetime']=data.index

    print(data.tail(1))
    
    time.sleep(0.5)